In [1]:
import hyperopt
import arff
import pandas as pd
import numpy as np

In [2]:
def split_data(X, y):
    np.random.seed(42)
    inds = np.random.permutation(np.arange(X.shape[0]))
    i1, i2 = (np.array([2/8., 3./8]) * X.shape[0]).astype(int)
    X_train, y_train = X[inds[:i1]], y[inds[:i1]]
    X_val, y_val = X[inds[i1:i2]], y[inds[i1:i2]]
    X_test, y_test = X[inds[i2:]], y[inds[i2:]]
    return X_train, X_val, X_test, y_train, y_val, y_test

In [3]:
from hyperopt import fmin, tpe
from hyperopt import hp
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier

def evaluate_sklearn(X_train, X_val, X_test, y_train, y_val, y_test):
    def create_sklearn_model(params):
        return RandomForestClassifier(n_estimators=100, **{k:int(v) for k, v in params.items()})
    space = {
        'max_depth': hp.qloguniform('max_depth', 0, 6, 1),
        'min_samples_split': hp.qloguniform('min_samples_split', 1, 6, 1),
        'min_samples_leaf': hp.qloguniform('min_samples_leaf', 0, 6, 1),
        'max_leaf_nodes': hp.qloguniform('max_leaf_nodes', 2, 10, 1),
    }
    best = fmin(lambda params: roc_auc_score(
                    y_val,
                    create_sklearn_model(params).fit(X_train, y_train).predict_log_proba(X_val)[:,1]),
                space, algo=tpe.suggest, max_evals=5)
    model = create_sklearn_model(best).fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
    return roc_auc_score(y_test, model.predict_log_proba(X_test)[:, 1])

In [4]:
import lightgbm as lgb

def evaluate_lgb(X_train, X_val, X_test, y_train, y_val, y_test):
    train_data = lgb.Dataset(X_train, label=y_train)

    space = {
        'num_boost_round': hp.qloguniform('num_boost_round', 3, 6, 1),
        'num_leaves': hp.qloguniform('num_leaves', 2, 10, 1),
        'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 1, 6, 1),
        'max_depth': hp.qloguniform('max_depth', 0, 6, 1),
    }

    def func(params):
        params = {k:int(v) for k, v in params.items()}
        num_boost_round = params.pop('num_boost_round')
        model = lgb.train(params, train_data, num_boost_round)
        return roc_auc_score(y_val, model.predict(X_val))

    params = fmin(func, space, algo=tpe.suggest, max_evals=5)
    params = {k:int(v) for k, v in params.items()}
    num_boost_round = params.pop('num_boost_round')
    model = lgb.train({k:int(v) for k, v in params.items()},
                      lgb.Dataset(np.concatenate([X_train, X_val]),
                                  label=np.concatenate([y_train, y_val])),
                      num_boost_round)
    return roc_auc_score(y_test, model.predict(X_test))


In [5]:
data = list(arff.load('/home/serj/Downloads/BayesianNetworkGenerator_tic-tac-toe.arff'))[1:]

input_columns = [
    'bottom-left-square',
    'bottom-middle-square',
    'bottom-right-square',
    'middle-left-square',
    'middle-middle-square',
    'middle-right-square',
    'top-left-square',
    'top-middle-square',
    'top-right-square',
]
target_column = 'Class'

input_encoding = {'b': 0, 'x': 1, 'o': 2}
target_encoding = {'negative': 0, 'positive': 1}

X4 = np.array([[input_encoding[row._data[k]] for k in input_columns]
             for row in data])
y4 = np.array([target_encoding[row._data[target_column]] for row in data])

In [6]:
data_tuple = split_data(X4, y4)
print evaluate_sklearn(*data_tuple)
print evaluate_lgb(*data_tuple)

0.761789946379
0.811586767622


In [ ]:
с использованием hyperopt (https://github.com/hyperopt/hyperopt/) реализовать сравнение 2 алгоритмов классификации

    sklearn.ensemble.RandomForestClassifier https://github.com/scikit-learn/scikit-learn/blob/8d072fb/sklearn/ensemble/forest.py#L744
    LightGBM (https://github.com/Microsoft/LightGBM)

датасеты:

    https://www.openml.org/d/1216
    https://www.openml.org/d/1241
    https://www.openml.org/d/4154
    https://www.openml.org/d/137
    https://www.openml.org/d/1178

(можно на свое усмотрение выбрать другие алгоритмы и датасеты)

схема сравнения:

    делим датасет на 3 части X_train, X_val, X_test в пропорциях 2:1:5
    подбираем оптимальные параметры классификатора на X_train, выбираем такой набор, который дает лучшее качество на X_val (используем hyperopt)
    с этим набором параметров обучаем на X_train + X_val и применяем к X_test - качество (ROC_AUC сохраняем в табличку)
    повторяем для всех датасетов и всех классификаторов
    в конце упражнения для каждого классификатора получаем набор значений ROC_AUC (5 штук), который рисуем на графике - а-ля (https://github.com/openml/study_example/blob/master/StudyTest.ipynb)
    весь код + тетрадки сохранить в виде репозитория на github, который запускается с помощью everware
